# Naive representation pipeline

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

In [10]:
import numpy as np
import sys
import struct

import qiskit
from qiskit import *
from qiskit.visualization import plot_histogram

import pennylane as qml
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer

## Generating an artificial image of the sky

In [11]:
#image of the sky filled with single precision complex floats
#pixels are set to low complex random values (image background) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype=complex)
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[4.41553215e+01+9.37772404e+01j 4.41553215e+01+9.37772404e+01j
  5.78722914e-03+6.81361413e-03j 4.41553215e+01+9.37772404e+01j]
 [4.41553215e+01+9.37772404e+01j 7.36448987e-03+5.17846301e-03j
  7.10335715e-03+4.92891279e-03j 9.62911201e-03+8.98273493e-03j]
 [1.80833179e-03+6.01629661e-03j 5.13319494e-03+1.34600094e-03j
  5.27209078e-03+2.69728243e-03j 8.69603582e-03+5.97537684e-03j]
 [4.41553215e+01+9.37772404e+01j 2.35026649e-03+5.10027464e-03j
  9.43633908e-03+8.33385268e-04j 7.88972659e-03+9.25953112e-03j]]


## Applying 2D Fourier transform (visibilities)

In [12]:
visibilities = np.fft.fft2(sky_image)
print(visibilities)

[[ 220.8470775 +468.94333406j  132.42758081+281.33383134j
    44.14366599 +93.76268755j  132.45276661+281.31109749j]
 [ 132.45473889+281.31807934j   44.15025701 +93.760857j
   -44.13849129 -93.76273243j   44.14754793 +93.76869265j]
 [  44.13824506 +93.76580571j  -44.1446986  -93.77921398j
  -132.45623208-281.30075679j  -44.15136691 -93.7707315j ]
 [ 132.44694505+281.3269206j    44.16499774 +93.76623293j
   -44.14707956 -93.78090561j   44.14918931 +93.77264865j]]


### Sanity check

In [13]:
img = np.fft.ifft2(visibilities)
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[4.41553215e+01+9.37772404e+01j 4.41553215e+01+9.37772404e+01j
  5.78722914e-03+6.81361413e-03j 4.41553215e+01+9.37772404e+01j]
 [4.41553215e+01+9.37772404e+01j 7.36448987e-03+5.17846301e-03j
  7.10335715e-03+4.92891279e-03j 9.62911201e-03+8.98273493e-03j]
 [1.80833179e-03+6.01629661e-03j 5.13319494e-03+1.34600094e-03j
  5.27209078e-03+2.69728243e-03j 8.69603582e-03+5.97537684e-03j]
 [4.41553215e+01+9.37772404e+01j 2.35026649e-03+5.10027464e-03j
  9.43633908e-03+8.33385268e-04j 7.88972659e-03+9.25953112e-03j]]

Reals MSE: 1.315378298764437e-29
Imaginaries MSE: 5.577287105625867e-29


## Classical data encoding/decoding

In [14]:
number_of_entries = visibilities.shape[0] * visibilities.shape[1]
number_of_bits_real = 32 #since single precision complex floats
number_of_bits_imag = 32
number_of_bits = number_of_bits_real + number_of_bits_imag
off_set = 0

qc = QuantumCircuit(number_of_entries*number_of_bits) #n bits encoded into n qbits (circuit family #1)
for i in range(0, visibilities.shape[0]):
    for j in range(0, visibilities.shape[1]):
            binary_real = float_to_bin_real(visibilities[i, j].real)
            binary_imag = float_to_bin_imag(visibilities[i, j].imag)
            binary = binary_real+binary_imag
            off_set = encoding2(qc, binary, off_set)

#measurement
qc.measure_all()
backend = Aer.get_backend('aer_simulator')
job = backend.run(qc, shots=1, memory=True)
output = job.result().get_memory()[0]
out = reverse(output)

#readout
chunks_real = []
chunks_imag = []
for i in range(0, number_of_entries):
    chunks_real.append(out[number_of_bits*i:(number_of_bits*i)+number_of_bits_real]) #real parts represented every 64 bits starting from the first one
    chunks_imag.append(out[(number_of_bits*i)+number_of_bits_imag:(number_of_bits*i)+number_of_bits_imag+number_of_bits_imag]) #imaginary parts represented every 64 bits starting after the first occurence of a real part  
readout = []
for i in range(0, len(chunks_real)):
    readout.append(complex(bin_to_float_real(chunks_real[i]), bin_to_float_imag(chunks_imag[i]))) 

#reshaping the readout vector into a nxn matrix
readout = np.array(readout).reshape(n , n)
print(readout)

[[ 220.84707642+468.94332886j  132.42758179+281.33383179j
    44.14366531 +93.76268768j  132.45277405+281.31109619j]
 [ 132.45474243+281.31808472j   44.15025711 +93.76085663j
   -44.13849258 -93.76273346j   44.14754868 +93.76869202j]
 [  44.13824463 +93.76580811j  -44.1446991  -93.77921295j
  -132.45623779-281.30075073j  -44.15136719 -93.77072906j]
 [ 132.44694519+281.32693481j   44.1649971  +93.76623535j
   -44.14707947 -93.78090668j   44.149189   +93.77265167j]]


## Applying 2D Inverse Fourier transform (+ fidelity test)

In [15]:
img = np.fft.ifft2(readout)
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[4.41553216e+01+9.37772422e+01j 4.41553221e+01+9.37772408e+01j
  5.78641891e-03+6.81447983e-03j 4.41553221e+01+9.37772417e+01j]
 [4.41553233e+01+9.37772396e+01j 7.36451149e-03+5.17821312e-03j
  7.10320473e-03+4.92835045e-03j 9.62948799e-03+8.98337364e-03j]
 [1.80816650e-03+6.01530075e-03j 5.13315201e-03+1.34372711e-03j
  5.27095795e-03+2.69651413e-03j 8.69607925e-03+5.97381592e-03j]
 [4.41553214e+01+9.37772391e+01j 2.34913826e-03+5.09953499e-03j
  9.43589211e-03+8.32796097e-04j 7.88903236e-03+9.25946236e-03j]]

Reals MSE: 5.069035062015934e-13
Imaginaries MSE: 1.1865125739449887e-12


## Utils

### Quantum

In [16]:
def encoding2(qc, binary, off_set):
    
    for i in range(0, len(binary)):
        qc.reset(off_set+i)

        if binary[i]=='1':
            qc.x(off_set+i)
    
    off_set += len(binary)

    return off_set

### Classical

In [17]:
#float to binary / binary to float
def float_to_bin_real(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def float_to_bin_imag(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float_real(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]
def bin_to_float_imag(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

#reverse a string (for the measurement step readout)
def reverse(string):
    string = string[::-1]
    return string

#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask